In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import time

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from glob import glob
# from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Model, regularizers
from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
import matplotlib.pyplot as plt
import cv2 as cv
import matplotlib.pyplot as plt

In [4]:
! unzip /content/train_.zip -d train 

Archive:  /content/train_.zip
  inflating: train/train_danilovski.csv  
  inflating: train/train_dorogomilovo.csv  
  inflating: train/train_hamovniki.csv  
  inflating: train/train_krasnay_presnya.csv  
  inflating: train/train_moscow_city.csv  
  inflating: train/train_uznoportovii.csv  
  inflating: train/train_zamoskvoreche.csv  


In [57]:
df = pd.read_csv('/content/train/train_danilovski.csv')

In [58]:
df

,Unnamed: 0,longitude,latitude,uptown,park_area,office_area,industrial_zone,tourist_area
0,0,55.715362,37.583333,0,0,1,0,0
1,1,55.715336,37.583378,0,0,1,0,0
2,2,55.715284,37.583378,0,0,1,0,0
3,3,55.715258,37.583333,0,0,1,0,0
4,4,55.715284,37.583288,0,0,1,0,0
...,...,...,...,...,...,...,...,...
361,361,55.715473,37.583615,0,0,1,0,0
362,362,55.715147,37.583615,0,0,1,0,0
363,363,55.714985,37.583333,0,0,1,0,0
364,364,55.715147,37.583051,0,0,1,0,0


In [59]:
def csv_preprocess(df):
    for csv_path in glob('/content/train/*.csv'):
      df_temp = pd.read_csv(csv_path)
      print(len(df_temp.index))
      df = pd.concat([df, df_temp], axis = 0, ignore_index=True)
    return df
    

In [60]:
df = csv_preprocess(df)

72
102
408
366
6
6
276
72


In [99]:
df = pd.read_csv('/content/train_15_points.csv')

In [100]:
df.drop('Unnamed: 0', axis = 1, inplace=True)


In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2820 entries, 0 to 2819
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   longitude        2820 non-null   float64
 1   latitude         2820 non-null   float64
 2   uptown           2820 non-null   int64  
 3   park_area        2820 non-null   int64  
 4   office_area      2820 non-null   int64  
 5   industrial_zone  2820 non-null   int64  
 6   tourist_area     2820 non-null   int64  
dtypes: float64(2), int64(5)
memory usage: 154.3 KB


In [102]:
# df.to_csv('train_6_points.csv')

In [103]:
df

,longitude,latitude,uptown,park_area,office_area,industrial_zone,tourist_area
0,55.715881,37.583333,0,1,0,0,0
1,55.715805,37.583619,0,1,0,0,0
2,55.715596,37.583828,0,1,0,0,0
3,55.715310,37.583904,0,1,0,0,0
4,55.715024,37.583828,0,1,0,0,0
...,...,...,...,...,...,...,...
2815,55.715028,37.583170,0,0,1,0,0
2816,55.715147,37.583051,0,0,1,0,0
2817,55.715310,37.583008,0,0,1,0,0
2818,55.715473,37.583051,0,0,1,0,0


In [104]:
df['office_area'] = df['office_area'].replace({1: 2, 0: 0})
df['industrial_zone'] = df['industrial_zone'].replace({1: 3, 0: 0})
df['tourist_area'] = df['tourist_area'].replace({1: 4, 0: 0})

In [105]:
df

,longitude,latitude,uptown,park_area,office_area,industrial_zone,tourist_area
0,55.715881,37.583333,0,1,0,0,0
1,55.715805,37.583619,0,1,0,0,0
2,55.715596,37.583828,0,1,0,0,0
3,55.715310,37.583904,0,1,0,0,0
4,55.715024,37.583828,0,1,0,0,0
...,...,...,...,...,...,...,...
2815,55.715028,37.583170,0,0,2,0,0
2816,55.715147,37.583051,0,0,2,0,0
2817,55.715310,37.583008,0,0,2,0,0
2818,55.715473,37.583051,0,0,2,0,0


In [106]:
df['labels'] = df.loc[:, ['park_area', 'office_area', 'industrial_zone', 'tourist_area']].sum(axis=1)

In [107]:
df['labels'].unique()

array([1, 3, 0, 4, 2])

In [108]:
X = df.loc[:, ['longitude', 'latitude']].values
y = df.loc[:, 'labels'].values

In [109]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(X_train)

# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)


In [122]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [123]:
y_pred = classifier.predict(X_test)

In [124]:
y_pred

array([0, 1, 1, 1, 1, 1, 2, 1, 0, 0, 3, 1, 0, 2, 0, 1, 0, 2, 1, 0, 0, 0,
       3, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 2, 0, 1, 0, 2, 1, 2, 0,
       1, 0, 2, 0, 0, 2, 0, 1, 4, 2, 0, 1, 1, 0, 0, 0, 1, 3, 1, 0, 1, 0,
       2, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 2, 2, 1, 1, 0, 3, 0, 1, 0,
       0, 2, 4, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 2, 0, 0,
       0, 0, 1, 4, 1, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 2,
       3, 2, 1, 1, 0, 0, 0, 2, 3, 0, 1, 3, 0, 0, 2, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 2, 1, 4, 2, 1, 2, 0, 0, 2, 1, 1, 0, 0, 0, 0, 1, 3, 0,
       2, 0, 0, 2, 3, 2, 0, 0, 2, 1, 0, 3, 2, 2, 1, 0, 0, 1, 2, 2, 3, 0,
       0, 0, 0, 0, 4, 2, 3, 1, 2, 3, 4, 2, 1, 0, 0, 0, 2, 0, 0, 4, 0, 1,
       0, 2, 0, 1, 1, 1, 2, 0, 4, 2, 0, 0, 1, 0, 2, 0, 0, 1, 3, 0, 4, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 2, 0, 2, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 2, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 2, 0, 0, 4, 2, 2, 1, 2, 1, 0, 1, 1,

In [125]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[100  44  35  11   9]
 [ 72  49  19   4   7]
 [ 46  46  24   6   1]
 [ 31   6   4   3   1]
 [ 22  10   7   7   0]]
              precision    recall  f1-score   support

           0       0.37      0.50      0.43       199
           1       0.32      0.32      0.32       151
           2       0.27      0.20      0.23       123
           3       0.10      0.07      0.08        45
           4       0.00      0.00      0.00        46

    accuracy                           0.31       564
   macro avg       0.21      0.22      0.21       564
weighted avg       0.28      0.31      0.29       564



In [161]:
dictionary_otrasl = {'Рестораны' : [2, 3],
                     'Бары' : [1, 4], 
                     'Ночные клубы' : [2, 3, 4],
                     'Столовые' : [2, 3, 5],
                     'Стрит-фуд' : [1, 3, 5],
                     'Предприятия быстрого обслуживания' : [2, 4],
                     'Кафетерии' : [0, 1, 2, 3, 4,],
                     'Кофейни' : [0, 1, 2, 4,],
                     'Магазины кулинарии' : [0, 2, 4],
                     'Розливной алкоголь' : [0],
                     'Магазины продовольственные/непродовольственные частного спроса' : [0],
                     'Магазины-склады' : [3],
                     'Палатки' : [0, 4],
                     'Минимаркеты' : [1, 4],
                     'Мелкие магазины, расположенные на территории предприятий для обсуживания работников' : [2, 3],
                     'Магазины специализированные продвольственные (рыба, мясо, вина,  и пр)' : [0],
                     'Магазины специализированные непродовольственные ' : [0, 3],
                     'Магазины прочей товарной специализации (книги, зоомагазины, цветы и др)' : [0, 4],
                     'Вендинговые автоматы' : [2, 4],
                     'Фудтрак' : [1, 4],
                     'Клиники' : [0, 2],
                     'Ветклиники' : [0, 2],
                     'Ветаптеки' : [0, 2],
                     'Аптеки' : [0, 2],
                     'Оптика' : [0, 2],
                     'Ортика' : [0, 2],
                     'Стоматология' : [0, 2],
                     'Медицинские центры косметологии и эстетической медицины (кабинет)' : [2, 3],
                     'Ателье' : [0],
                     'Ремонт обуви' : [0],
                     'Изготовление ключей' : [0],
                     'Салоны красоты' : [0, 4],
                     'Фото' : [0, 2, 4],
                     'Ремонт техники' : [0, 3],
                     'Ремонт ювелирных изделий ' : [0],
                     'Квесты' : [3],
                     'Антикафе' : [4],
                     'Фитнесклубы' : [0, 2],
                     'Хобби' : [0, 2],
                     'Образовательные услуги (частное образование)' : [0, 2],
                     'Развивательные центры для детей' : [1, 3],
                     'Банковская сфера(филиалы)' : [0, 2, 4],
                     'Микрофинансовые организации' : [0, 2],
                     'Нотариус' : [0, 2],
                     'Адвокатская контора' : [0, 2],
}

In [162]:
list_category = list(dictionary_otrasl.keys())

In [163]:
list_category

['Рестораны',
 'Бары',
 'Ночные клубы',
 'Столовые',
 'Стрит-фуд',
 'Предприятия быстрого обслуживания',
 'Кафетерии',
 'Кофейни',
 'Магазины кулинарии',
 'Розливной алкоголь',
 'Магазины продовольственные/непродовольственные частного спроса',
 'Магазины-склады',
 'Палатки',
 'Минимаркеты',
 'Мелкие магазины, расположенные на территории предприятий для обсуживания работников',
 'Магазины специализированные продвольственные (рыба, мясо, вина,  и пр)',
 'Магазины специализированные непродовольственные ',
 'Магазины прочей товарной специализации (книги, зоомагазины, цветы и др)',
 'Вендинговые автоматы',
 'Фудтрак',
 'Клиники',
 'Ветклиники',
 'Ветаптеки',
 'Аптеки',
 'Оптика',
 'Ортика',
 'Стоматология',
 'Медицинские центры косметологии и эстетической медицины (кабинет)',
 'Ателье',
 'Ремонт обуви',
 'Изготовление ключей',
 'Салоны красоты',
 'Фото',
 'Ремонт техники',
 'Ремонт ювелирных изделий ',
 'Квесты',
 'Антикафе',
 'Фитнесклубы',
 'Хобби',
 'Образовательные услуги (частное образ

In [209]:
df_labels = pd.read_excel('/content/label_time.xlsx', header=None)
df_labels.columns = ['6-10', '10-12', '12-15', '15-17', '17-20', '20-23', '23-3', '3-6']
df_labels.index = list_category
df_labels ['labels'] = dictionary_otrasl.values()
df_labels


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,6-10,10-12,12-15,15-17,17-20,20-23,23-3,3-6,labels
Рестораны,0,0,1,0,1,1,1,0,"[2, 3]"
Бары,0,0,0,0,1,1,1,0,"[1, 4]"
Ночные клубы,0,0,0,0,1,1,1,0,"[2, 3, 4]"
Столовые,0,0,1,1,0,0,0,0,"[2, 3, 5]"
Стрит-фуд,0,1,1,1,1,1,0,0,"[1, 3, 5]"
Предприятия быстрого обслуживания,0,1,1,1,1,1,0,0,"[2, 4]"
Кафетерии,1,1,1,1,1,1,0,0,"[0, 1, 2, 3, 4]"
Кофейни,1,1,1,1,1,1,0,0,"[0, 1, 2, 4]"
Магазины кулинарии,1,1,1,1,1,1,0,0,"[0, 2, 4]"
Розливной алкоголь,0,0,0,0,1,1,0,0,[0]


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [210]:
df_labels['labels']

Рестораны                                                                                       [2, 3]
Бары                                                                                            [1, 4]
Ночные клубы                                                                                 [2, 3, 4]
Столовые                                                                                     [2, 3, 5]
Стрит-фуд                                                                                    [1, 3, 5]
Предприятия быстрого обслуживания                                                               [2, 4]
Кафетерии                                                                              [0, 1, 2, 3, 4]
Кофейни                                                                                   [0, 1, 2, 4]
Магазины кулинарии                                                                           [0, 2, 4]
Розливной алкоголь                                                       

In [211]:
df_labels.to_csv('df_labels.csv')